In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2 # load images 
from tensorflow import keras
import tensorflow as tf
# Input data files are available in the read-only "../input/" directory
from tqdm import tqdm 
from pathlib import Path
pwd = %pwd
pwd = Path(pwd)
list(pwd.iterdir())

# Unzipping the Files

In [4]:
import zipfile as zp
with zp.ZipFile("/kaggle/input/dogs-vs-cats/test1.zip","r") as z:
    z.extractall(".")
with zp.ZipFile("/kaggle/input/dogs-vs-cats/train.zip","r") as z:
    z.extractall(".")

# Locating file path

In [5]:
train_path = Path(pwd) / 'train'
for index,file_item in enumerate(train_path.iterdir()):
    print(file_item)
    if index == 4:
        break

# Creating DataFrane for training 

In [6]:
img_data = pd.DataFrame(columns = ['path', 'labels'])
# w = []
# h = []
for i in tqdm(train_path.iterdir()):
    idx = len(img_data.index)
    img_data.loc[idx, 'path'] = str(i)
    img_data.loc[idx,'labels'] =  str(i).split('/')[-1][:3]
    img_shape = cv2.imread(str(i),0).shape
#     w.append(img_shape[0])
#     h.append(img_shape[1])

## Choosing image size

In [ ]:
print(np.array(w).mean())
print(np.array(h).mean()) 


on an average values are between 360 to 400 So let's take 350 as it is close to both the values 

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Creating Generator for training

In [8]:
traning_generator = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split = 0.2,
)


## Defining Flow for training

In [9]:
train_generator = traning_generator.flow_from_dataframe(
    dataframe = img_data,
     x_col='path',
    y_col='labels',
    target_size=(350, 350),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset= 'training'
)

val_generator = traning_generator.flow_from_dataframe(
    dataframe = img_data,
     x_col='path',
    y_col='labels',
    target_size=(350, 350),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset= 'validation'
)


# Initializing ImageDataGenerator

In [10]:
traning_generator = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split = 0.2,
)


In [11]:
train_generator = traning_generator.flow_from_dataframe(
    dataframe = img_data,
     x_col='path',
    y_col='labels',
    target_size=(350, 350),
    color_mode='rgb',
    class_mode='binary',
    batch_size=64,
    shuffle=True,
    seed=42,
    subset= 'training'
)

val_generator = traning_generator.flow_from_dataframe(
    dataframe = img_data,
     x_col='path',
    y_col='labels',
    target_size=(350, 350),
    color_mode='rgb',
    class_mode='binary',
    batch_size=64,
    shuffle=True,
    seed=42,
    subset= 'validation'
)

# Model Creation Step

In [12]:
from tensorflow.keras.models import Sequential

In [13]:
from tensorflow.keras.layers import Dense,Conv2D,GlobalAveragePooling2D,Flatten,Dropout,MaxPool2D

In [2]:
model = keras.models.Sequential()

model.add(Conv2D(filters=32,kernel_size=(3,3),input_shape=(350,350,3),activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))

model.add(GlobalAveragePooling2D())

model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'],
)
model.summary()

In [16]:
model.fit(
    train_generator,
    validation_data = val_generator,
    epochs = 20,
    callbacks = [
                 keras.callbacks.EarlyStopping(
                     monitor = 'val_loss',
                     patience = 5,
                     restore_best_weights = True
                 ),

                 keras.callbacks.ReduceLROnPlateau(
                      monitor='val_loss',
                      patience=3
                )
    ]
)

In [ ]:
results = model.evaluate(test_generator, verbose=0)

print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

In [ ]:
predictions = (model.predict(test_generator) >= 0.5).astype(np.int)

cm = confusion_matrix(test_generator.labels, predictions, labels=[0, 1])
clr = classification_report(test_generator.labels, predictions, labels=[0, 1], target_names=["CAT", "DOG"])

plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='g', vmin=0, cmap='Blues', cbar=False)
plt.xticks(ticks=[0.5, 1.5], labels=["CAT", "DOG"])
plt.yticks(ticks=[0.5, 1.5], labels=["CAT", "DOG"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

print("Classification Report:\n----------------------\n", clr)